In [1]:
include("src/Device.jl")
include("src/ReadIn.jl")

excitation_csv = "data/常见染料_发射曲线.csv"
emission_csv   = "data/常见染料_激发曲线.csv"
antigens_csv   = "data/使用抗体.csv";

In [2]:
@time (metadata, template, interaction) = ReadIn.get_normalized_data(excitation_csv, emission_csv, antigens_csv, 
    Device.ThermoFisher);

data restored from: 69e1650b35c4814f2864dc3dc0c1ee
  7.335196 seconds (6.77 M allocations: 355.449 MiB, 2.38% gc time)


In [5]:
# NOTE: this step shall be done manually or using Interactive module.
readin_marker_density = [
    (:PDL_1, 100), 
    (:IgD,   20,  :biotin), 
    (:Bcl_6, 4000)
]

function get_marker_density(markers, readin_marker_density; default_value=1, default_flag=:normal)
    alpha_dict = Dict(:normal=>1, :biotin=>4)
    result = Dict{Symbol, Float64}()
    for item in markers
        result[item] = default_value*alpha_dict[default_flag]
    end
    for item in readin_marker_density
        if length(item) == 2
            result[item[1]] = item[2]*alpha_dict[default_flag]
        elseif length(item) == 3
            result[item[1]] = item[2]*alpha_dict[item[3]]
        else
            throw(BoundsError("readin_marker_density have invalid shape. require: Array{Tuple{Symbol, Float64[, Symbol]}}"))
        end
    end
    [(name, result[name]) for name in markers]
end

marker_density = get_marker_density(metadata.markers, readin_marker_density)

41-element Array{Tuple{Symbol,Float64},1}:
 (:Bcl_6, 4000.0)  
 (:CD11c, 1.0)     
 (:GL_7, 1.0)      
 (:CD8alpha, 1.0)  
 (:B220, 1.0)      
 (:CD3, 1.0)       
 (:CD45_1, 1.0)    
 (:CD45_2, 1.0)    
 (:CD8, 1.0)       
 (:CD80, 1.0)      
 (:F4_80, 1.0)     
 (:FAS, 1.0)       
 (:IgG, 1.0)       
 ⋮                 
 (:TCR_beta, 1.0)  
 (:Thy1_2, 1.0)    
 (:AnnexinV, 1.0)  
 (:CD138, 1.0)     
 (:CD38, 1.0)      
 (:CD86, 1.0)      
 (:CXCR4, 1.0)     
 (:TCRValpha2, 1.0)
 (:Thy1_1, 1.0)    
 (:PD_1, 1.0)      
 (:PDL_1, 100.0)   
 (:CD205, 1.0)     

In [6]:
initiation = begin
    template_n = 0
    demo_template = template
    for (each_laser, ports) in Device.ThermoFisher.lens
        for (each_port, _) in ports
            template_n += 1
            
            for i=1:length(metadata.markers)
                demo_template[i,:,template_n] *= log(marker_density[i][2])
            end
            
            for i=1:length(metadata.dyes)
                dyename = metadata.dyes[i].name
                
                demo_template[:,i,template_n] *= 2^metadata.dyes[i].brightness *
                    metadata.exratio[each_laser][1, dyename]/100 * 
                    mean(metadata.emratio[each_laser][each_port][:, dyename])
            end
        end
    end
    demo_template[isnan.(demo_template)]=-999
    demo_template
end;

In [7]:
function findmax_pos(ndarray, n=3)
    value, pos = findmax(ndarray)
    value,(pos % size(ndarray)[1], 
        ceil(pos/size(ndarray)[1]) % size(ndarray)[2] |> Int, 
        ceil(pos/size(ndarray)[1]/size(ndarray)[2]) % size(ndarray)[1] |> Int )
end

findmax_pos (generic function with 2 methods)

In [8]:
result = Dict{Symbol, Tuple{Symbol, Float64}}()
choice = :unkown
for each_epoch in readin_marker_density
    print(each_epoch)
    value, pos = findmax_pos(template)
    print(value," ",pos, " ")
    choicer = metadata.dyes[pos[2]].name
    choicee = metadata.markers[pos[1]]
    result[choicee] = (choicer, value)
    println(choicee," ",choicer)
    
    initiation[:,pos[2],:] = NaN
    initiation[pos[1],:,:] = NaN
    
    template_n = 0
    for (each_laser, ports) in Device.ThermoFisher.lens
        for (each_port, _) in ports
            template_n += 1
            applyer = interaction[each_laser][each_port][choicer]
            initiation[:,:,template_n] .*= applyer'
        end
    end
end
initiation

(:PDL_1, 100)783284.2722558198 (1, 3, 1) Bcl_6 Alexa647
(:IgD, 20, :biotin)32635.75938529558 (40, 9, 1) PDL_1 PE_Cy7
(:Bcl_6, 4000)83.8441790858306 (27, 8, 4) IgD PE


41×12×18 Array{Float64,3}:
[:, :, 1] =
 NaN    NaN    NaN  NaN    NaN    NaN    NaN    NaN  NaN  NaN    NaN    NaN  
  -0.0    0.0  NaN   -0.0   -0.0   -0.0   -0.0  NaN  NaN   -0.0   -0.0   -0.0
  -0.0    0.0  NaN   -0.0   -0.0   -0.0   -0.0  NaN  NaN   -0.0   -0.0   -0.0
  -0.0    0.0  NaN   -0.0   -0.0   -0.0   -0.0  NaN  NaN   -0.0   -0.0   -0.0
  -0.0    0.0  NaN   -0.0   -0.0   -0.0   -0.0  NaN  NaN   -0.0   -0.0   -0.0
  -0.0    0.0  NaN   -0.0   -0.0   -0.0   -0.0  NaN  NaN   -0.0   -0.0   -0.0
  -0.0    0.0  NaN   -0.0   -0.0   -0.0   -0.0  NaN  NaN   -0.0   -0.0   -0.0
  -0.0    0.0  NaN   -0.0   -0.0   -0.0   -0.0  NaN  NaN   -0.0   -0.0   -0.0
  -0.0    0.0  NaN   -0.0   -0.0   -0.0   -0.0  NaN  NaN   -0.0   -0.0   -0.0
  -0.0    0.0  NaN   -0.0   -0.0   -0.0   -0.0  NaN  NaN   -0.0   -0.0   -0.0
  -0.0    0.0  NaN   -0.0   -0.0   -0.0   -0.0  NaN  NaN   -0.0   -0.0   -0.0
  -0.0    0.0  NaN   -0.0   -0.0   -0.0   -0.0  NaN  NaN   -0.0   -0.0   -0.0
  -0.0    0.0  NaN   -0.0

In [9]:
result

Dict{Symbol,Tuple{Symbol,Float64}} with 3 entries:
  :PDL_1 => (:PE_Cy7, 32635.8)
  :Bcl_6 => (:Alexa647, 7.83284e5)
  :IgD   => (:PE, 83.8442)